# 🛣️ Week 07-08 · Notebook 08 · Query Optimization & Routing

Design routing logic that directs technician queries to the best retrieval path or tool while enforcing policy.

## 🎯 Learning Objectives
- Implement classifier-based routing using LangChain.
- Build conditional pipelines that swap prompts/tools per plant and language.
- Enforce risk-based routing (e.g., EHS incidents escalate to SME).
- Log routing decisions for compliance.

## 🧩 Scenario
Bilingual technicians submit tickets. The system must detect Spanish queries, route to translation-enabled retrieval, and escalate high-risk EHS issues directly to human review.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch

# --- 1. Define the different "expert" chains ---

# Prompt for a maintenance-focused assistant
maintenance_template = """You are an expert in mechanical maintenance. 
Answer the following question concisely based on standard procedures.
Question: {question}"""
maintenance_prompt = ChatPromptTemplate.from_template(maintenance_template)

# Prompt for a quality control-focused assistant
quality_template = """You are an expert in quality control and inspection. 
Answer the following question with a focus on ISO 9001 compliance.
Question: {question}"""
quality_prompt = ChatPromptTemplate.from_template(quality_template)

# LLM for the expert chains
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.1)

# Build the expert chains
maintenance_chain = maintenance_prompt | llm | StrOutputParser()
quality_chain = quality_prompt | llm | StrOutputParser()


# --- 2. Define the Routing Logic ---

# A prompt to classify the user's question
routing_template = """Given the user question below, classify it as either 'maintenance' or 'quality'.
Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
routing_prompt = ChatPromptTemplate.from_template(routing_template)

# The classification chain
classification_chain = routing_prompt | llm | StrOutputParser()


# --- 3. Create the RunnableBranch ---
# This is the core of the routing logic. It directs the input to the correct chain.
# It takes a list of (condition, runnable) pairs and a default runnable.
# The condition is a function that returns True or False.
branch = RunnableBranch(
    (lambda x: "maintenance" in x["topic"].lower(), maintenance_chain),
    (lambda x: "quality" in x["topic"].lower(), quality_chain),
    maintenance_chain  # Default chain
)

# --- 4. Combine into a full chain ---
# The full chain first classifies the topic, then the branch executes the correct expert chain.
full_chain = {"topic": classification_chain, "question": lambda x: x["question"]} | branch

# --- 5. Execute the router ---
question_mq = "The surface finish on part #789 is out of spec. What should be the inspection process?"
question_maint = "Los sensores registran temperatura alta en el husillo." # "The sensors are recording high temperature in the spindle."

print("--- Running Quality Control Query ---")
print(f"Question: {question_mq}")
result_qc = full_chain.invoke({"question": question_mq})
print(f"Answer:\n{result_qc}\n")

print("--- Running Maintenance Query ---")
print(f"Question: {question_maint}")
result_maint = full_chain.invoke({"question": question_maint})
print(f"Answer:\n{result_maint}")

### ⚠️ Risk Routing
Define policy: if query matches EHS keywords ("hazard", "injury"), bypass automation and escalate to SME while providing immediate safety guidance.

In [ ]:
# Define keywords that trigger an escalation to a human SME (Subject Matter Expert)
EHS_TERMS = {'hazard', 'injury', 'fatal', 'spill', 'arc flash', 'fire', 'emergency'}

def determine_risk_route(query: str) -> str:
    """
    Determines the routing path based on query content.
    'sme_escalation' for high-risk EHS terms, 'standard_rag' otherwise.
    """
    if any(term in query.lower() for term in EHS_TERMS):
        return 'sme_escalation'
    return 'standard_rag'

# --- Example Usage ---
queries = [
    'Arc flash detected near Panel 4. What is the emergency procedure?',
    'Necesitamos calibrar el husillo.', # "We need to calibrate the spindle."
    'There is a small chemical spill in the plating area.'
]

routes = {q: determine_risk_route(q) for q in queries}

for query, route in routes.items():
    print(f"Query: '{query}'\nRoute: {route}\n")

# In a full application, you would use another RunnableBranch here.
# For example:
#
# sme_chain = (lambda x: "EMERGENCY: Human intervention required for query: " + x["question"])
# risk_branch = RunnableBranch(
#     (lambda x: determine_risk_route(x["question"]) == 'sme_escalation', sme_chain),
#     full_chain # The standard RAG/expert chain from before
# )
#
# print(risk_branch.invoke({"question": "There is a fire in the warehouse!"}))

## 🧪 Lab Assignment
1. Integrate retrieval chains per route and benchmark routing accuracy.
2. Store routing decision logs in BigQuery with user ID, timestamp, risk level.
3. Add A/B test toggles to evaluate new routing classifiers.
4. Present routing coverage metrics to the safety council.

## ✅ Checklist
- [ ] Routing logic covers bilingual queries
- [ ] EHS escalation paths documented
- [ ] Logs captured for audits
- [ ] Lab deliverables accepted

## 📚 References
- LangChain Router Chains
- Corporate EHS Escalation SOP
- Week 05 Data Governance Notebook